In [ ]:
from typing import List

from plot_overlap_underground_lines import plot_overlap_path
from build_data import Station, build_data
import argparse
import heapq
from anytree import Node, RenderTree
import csv
import math
import sys
from geopy.distance import geodesic
import time

from plot_underground_path import plot_path


def Dijkstra(current, neighbor, g_score, end_station):
    """
    我们知道在极端情况下，当启发函数h(n)始终为0，则将由g(n)决定节点的优先级，此时算法就退化成了Dijkstra算法
    """
    # 距离起点的代价
    tentative_g_score = g_score[current.name] + Euclidean_distance(current.name, neighbor.name,
                                                                   stations)  # 前面没两站之间的g_score+这次的

    # h_score：距离终点的代价，启发函数
    h_score = 0
    # 总代价
    f_score = tentative_g_score + h_score
    return tentative_g_score, f_score


def best_first_search(current, neighbor, g_score, end_station):
    """
    在另外一个极端情况下，如果h(n)相较于g(n)大很多，则此时只有h(n)产生效果，这也就变成了最佳优先搜索。
    这里我们将h(n)乘上1000
    """
    # 距离起点的代价
    tentative_g_score = g_score[current.name] + Euclidean_distance(current.name, neighbor.name,
                                                                   stations)  # 前面没两站之间的g_score+这次的

    # h_score：距离终点的代价，启发函数
    h_score = Euclidean_distance(neighbor.name, end_station.name, stations) * 1000
    # 总代价
    f_score = tentative_g_score + h_score
    return tentative_g_score, f_score


def f_score_Euclidean_distance(current, neighbor, g_score, end_station):
    """
    根据欧几里得距离算出来的f_score
    goal：距离最短
    参数：current：当前的站点
         neighbor：相邻的站点
    """
    # 距离起点的代价
    tentative_g_score = g_score[current.name] + Euclidean_distance(current.name, neighbor.name,
                                                                   stations)  # 前面没两站之间的g_score+这次的

    # h_score：距离终点的代价，启发函数
    h_score = Euclidean_distance(neighbor.name, end_station.name, stations)
    # 总代价
    f_score = tentative_g_score + h_score
    return tentative_g_score, f_score


# 标准的曼哈顿距离
def f_score_manhattan_distance(current, neighbor, g_score, end_station):
    """
    曼哈顿距离
    """
    # g_score: the cost from the start position to now
    dx_current_neighbor = abs(current.position[0] - neighbor.position[0])
    dy_current_neighbor = abs(current.position[1] - neighbor.position[1])
    tentative_g_score = g_score[current.name] + dx_current_neighbor + dy_current_neighbor
    # h_score：距离终点的代价，启发函数
    dx_current_end = abs(current.position[0] - end_station.position[0])
    dy_current_end = abs(current.position[1] - end_station.position[1])
    h_score = dx_current_end + dy_current_end
    # f_score
    f_score = tentative_g_score + h_score
    return tentative_g_score, f_score


# 不使用abs()，更倾向于向45度方向寻找
def f_score_manhattan_distance_45_degree(current, neighbor, g_score, end_station):
    # g_score: the cost from the start position to now
    dx_current_neighbor = current.position[0] - neighbor.position[0]
    dy_current_neighbor = current.position[1] - neighbor.position[1]
    tentative_g_score = g_score[current.name] + dx_current_neighbor + dy_current_neighbor
    # h_score：距离终点的代价，启发函数
    dx_current_end = current.position[0] - end_station.position[0]
    dy_current_end = current.position[1] - end_station.position[1]
    h_score = dx_current_end + dy_current_end
    # f_score
    f_score = tentative_g_score + h_score
    return tentative_g_score, f_score


# 不使用abs()，更倾向于向从起点到终点的方向寻找
def f_score_manhattan_distance_definite_degree(current, neighbor, g_score, end_station, degree):
    # g_score: the cost from the start position to now
    dx_current_neighbor = current.position[0] - neighbor.position[0]
    # 期望的y
    y_hat_current_neighbor = current.position[1] + dx_current_neighbor * math.tan(degree)
    # 实际的y
    y_true_current_neighbor = neighbor.position[1]
    # dy越接近0越好
    dy_current_neighbor = abs(y_hat_current_neighbor - y_true_current_neighbor)
    # 得到g_score
    tentative_g_score = g_score[current.name] + dy_current_neighbor

    # h_score：距离终点的代价，启发函数
    dx_end_neighbor = end_station.position[0] - neighbor.position[0]
    y_hat_end_neighbor = neighbor.position[1] + dx_end_neighbor * math.tan(degree)
    y_true_end_neighbor = end_station.position[1]
    dy_end_neighbor = abs(y_hat_end_neighbor - y_true_end_neighbor)
    h_score = dy_end_neighbor

    # f_score
    f_score = tentative_g_score + h_score

    return tentative_g_score, f_score


# 还没具体实现###############################################################################################
def f_score_diagonal_distance(current, neighbor, g_score, end_station):
    # g_score
    dx_g = abs(neighbor.position[0] - current.position[0])
    dy_g = abs(neighbor.position[1] - current.position[1])
    tentative_g_score = g_score[current.name] + max(dx_g, dy_g) + (math.sqrt(2) - 1) * min(dx_g, dy_g)
    # 启发函数
    dx_h = abs(neighbor.position[0] - end_station.position[0])
    dy_h = abs(neighbor.position[1] - end_station.position[1])
    h_score = max(dx_h, dy_h) + (math.sqrt(2) - 1) * min(dx_h, dy_h)
    # f_score
    f_score = tentative_g_score + h_score

    return tentative_g_score, f_score


def f_score_Haversine_Distance(current, neighbor, g_score, end_station):
    """
    position[0]：维度；position[1]：经度
    """
    lat1 = current.position[0]
    lat2 = neighbor.position[0]
    det_lat = current.position[0] - neighbor.position[0]
    det_lon = current.position[1] - neighbor.position[1]
    R = 6371
    a = (math.sin(det_lat / 2)) ** 2 + math.cos(lat1) * math.cos(lat2) * (math.sin(det_lon / 2)) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    tentative_g_score = g_score[current.name] + d

    # 启发函数
    lat1 = neighbor.position[0]
    lat2 = end_station.position[0]
    det_lat = neighbor.position[0] - end_station.position[0]
    det_lon = neighbor.position[1] - end_station.position[1]
    R = 6371
    a = (math.sin(det_lat / 2)) ** 2 + math.cos(lat1) * math.cos(lat2) * (math.sin(det_lon / 2)) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    h_score = d

    f_score = tentative_g_score + h_score
    return tentative_g_score, f_score


def f_score_geodesic_distance(current, neighbor, g_score, end_station):
    """
    Haversine distance 是通过 Haversine 公式计算的球面距离。Haversine 公式适用于球体，它假设地球是一个完美的球体，因此对于较短的距离来
    说是准确的。然而，随着距离增加，Haversine 公式可能会引入一些误差，因为地球的形状实际上更接近于椭球体。
    Geodesic distance 考虑了地球的实际形状，即椭球体，以提供更准确的距离测量。Vincenty 公式是一种常用于计算椭球体上两点之间 geodesic
    distance 的方法。
    """
    g_distance = geodesic(current.position, neighbor.position).kilometers
    tentative_g_score = g_score[current.name] + g_distance

    h_distance = geodesic(neighbor.position, end_station.position).kilometers
    h_score = h_distance

    f_score = tentative_g_score + h_score
    return tentative_g_score, f_score


# 还没具体实现################################################################################################


def f_score_NumberOfTransfers():
    """
    根据换乘次数算出来的f_score
    goal：换乘次数最少
    """
    pass


def find_paths(node, target_leaf_name, path=[]):
    """
    node：根节点（Node类型）
    target_leaf_name：输入名称即可
    寻找所有叶子节点为目标的支路并打印路线上的所有节点
    返回：返回一个大list，包含许多小list代表每条路径
    """
    all_paths = []
    path = path + [node.name]
    if node.is_leaf and node.name == target_leaf_name:
        all_paths.append(path.copy())
    for child in node.children:
        all_paths.extend(find_paths(child, target_leaf_name, path))
    return all_paths


def Calculate_distance(path, map: dict[str, Station]) -> List[str]:
    """
    path：一个list，包含该条路线上所有站点Station对象
    goal：给定一条路线，计算该条路线的总距离
    """
    total_distance = 0
    path1 = path[0:len(path) - 1]  # path中第一个到倒数第二个元素
    path2 = path[1:len(path)]  # path中第二个到最后一个元素
    for i in range(len(path1)):  # len(path1) = len(path2)
        station_name_1 = path1[i]
        station_name_2 = path2[i]
        distance = Euclidean_distance(station_name_1, station_name_2, stations)
        total_distance = total_distance + distance
    return total_distance


def Euclidean_distance(start_station_name: str, end_station_name: str, map: dict[str, Station]) -> List[str]:
    """
    创建一个叫Euclidean_distance方法：
    goal：计算任意两站点之间的欧几里得距离
    input：任意两个站点的name（Station.name）
    output：根据position（坐标）计算出的两站点之间的欧几里得距离
    """
    start_station = map[start_station_name]
    end_station = map[end_station_name]
    return ((start_station.position[0] - end_station.position[0]) ** 2 +
            (start_station.position[1] - end_station.position[1]) ** 2) ** 0.5


# Implement the following function
def get_path(start_station_name: str, end_station_name: str, map: dict[str, Station]) -> List[str]:
    """
    runs astar on the map, find the shortest path between a and b
    Args:
        start_station_name(str): The name of the starting station
        end_station_name(str): str The name of the ending station
        map(dict[str, Station]): Mapping between station names and station objects of the name,
                                 Please refer to the relevant comments in the build_data.py
                                 for the description of the Station class
    Returns:
        List[Station]: A path composed of a series of station_name
    """
    # You can obtain the Station objects of the starting and ending station through the following code
    start_station = map[start_station_name]  # 是Station类型
    end_station = map[end_station_name]
    # start working

    # 为”更倾向于从起点指向终点方向寻找的曼哈顿距离“服务的参数
    tan = (end_station.position[1] - start_station.position[1]) / (end_station.position[0] - start_station.position[0])
    degree = math.atan(tan)

    nodes = {}  # 用于存储站点名称与对应Node对象的映射
    root = Node(start_station)
    nodes[start_station.name] = root
    # 实现A*搜索算法
    open_set = []  # 初始化一个空的优先队列
    heapq.heappush(open_set, (0, start_station))  # 将起始站点添加到优先队列
    close_set = []  # 用于记录路径
    # 遍历all_stations列表中的每一个元素，然后把每一个元素作为键（key），对应的初始值设置为无穷大（float('inf')）。
    g_score = {station: float('inf') for station in stations}  # 初始化所有站点的g_score为无限大-inf
    # print(g_score)-output:{'Acton Town': inf, 'Aldgate': inf,......}
    g_score[start_station.name] = 0  # 起始站点的g_score为0
    # 一直执行，直到open_set变为空
    while open_set:
        # 前面定义的是(0, start_station)，所以返回[1]代表Station对象。[0]返回的是当前的优先级
        current = heapq.heappop(open_set)[1]  # 从优先队列中取出当前站点，Station对象
        if current == end_station:  # 如果当前站点是目标站点，重构并返回路径
            # 打印树
            # for pre, fill, node in RenderTree(root):
            #     print(f"{pre}{node.name}")
            path = []
            target_node = nodes[current.name]  # 找到目标节点的所有父节点
            ancestors = target_node.ancestors
            for ancestor in ancestors:
                # type(ancestor)-<class 'anytree.node.node.Node'>; type(ancestor.name))-<class 'build_data.Station'>
                path.append(ancestor.name.name)
            path.append(current.name)  # 把终点站加进去
            return path  # 返回path
        else:  # 如果当前站点不是终点
            close_set.append(current)
            for neighbor in current.links:  # 遍历当前与当前Station对象相邻的所有Station对象
                if neighbor in close_set:  # 防止走回头路
                    continue
                else:
                    nodes[neighbor.name] = Node(neighbor, parent=nodes[current.name])

                    # 对应不同的启发函数################################################################################
                    ################################################################################################
                    # 欧几里得距离
                    # tentative_g_score, f_score = f_score_Euclidean_distance(current, neighbor, g_score, end_station)

                    # 标准的曼哈顿距离
                    # tentative_g_score, f_score = f_score_manhattan_distance(current, neighbor, g_score, end_station)

                    # 更倾向于向45度方向寻找的曼哈顿距离
                    # tentative_g_score, f_score = f_score_manhattan_distance_45_degree(current, neighbor, g_score,
                    #                                                                   end_station)

                    # 更倾向于从起点指向终点方向寻找的曼哈顿距离
                    # tentative_g_score, f_score = f_score_manhattan_distance_definite_degree(current, neighbor, g_score,
                    #                                                                         end_station, degree)

                    # 对角距离 报错 还未解决
                    # tentative_g_score, f_score = f_score_diagonal_distance(current, neighbor, g_score, end_station)

                    # Haversine distance
                    # tentative_g_score, f_score = f_score_Haversine_Distance(current, neighbor, g_score, end_station)

                    # Geodesic distance
                    # tentative_g_score, f_score = f_score_geodesic_distance(current, neighbor, g_score, end_station)

                    # Dijkstra
                    # tentative_g_score, f_score = Dijkstra(current, neighbor, g_score, end_station)

                    # best_first search
                    tentative_g_score, f_score = best_first_search(current, neighbor, g_score, end_station)
                    # over##########################################################################################

                    if tentative_g_score < g_score[neighbor.name]:  # 确保g_socre不同的两天路线不会交叉在同一个站点
                        # 如果从当前站点到邻接站点的g_score更小
                        g_score[neighbor.name] = tentative_g_score
                        heapq.heappush(open_set, (f_score, neighbor))  # 将邻接站点加入优先队列
    return []  # 找不到就返回空列表


if __name__ == '__main__':
    # 创建ArgumentParser对象
    parser = argparse.ArgumentParser()
    # 添加命令行参数
    parser.add_argument('start_station_name', type=str, help='start_station_name')
    parser.add_argument('end_station_name', type=str, help='end_station_name')
    args = parser.parse_args()
    start_station_name = args.start_station_name
    end_station_name = args.end_station_name
    # The relevant descriptions of stations and underground_lines can be found in the build_data.py
    stations, underground_lines = build_data()
    # print(stations)-返回{'Acton Town': <build_data.Station object at 0x000001915B0B7760>,......}
    # 计时开始##############################################################################################
    start_time = time.time()
    #######################################################################################################
    path = get_path(start_station_name, end_station_name, stations)
    # 记录结束时间############################################################################################
    end_time = time.time()
    # 计算经过的时间
    elapsed_time = end_time - start_time
    print(f"The elapsed time is {elapsed_time} seconds.")
    ########################################################################################################
    # test #################################################################################################
    print("The path form", start_station_name, "to", end_station_name, "is", path)
    distance = Calculate_distance(path, stations)
    print("The total distance of this path is:", distance)
    # 定义输出文件名##############################################################################
    output_file = 'output.txt'
    # 打开文件并将 sys.stdout 重定向到文件（以 'a' 模式打开，表示追加内容）
    with open(output_file, 'a') as f:
        original_stdout = sys.stdout  # 保存原始的 sys.stdout
        # 将输出重定向到文件
        sys.stdout = f
        # 执行你的代码，这里是一个简单的输出
        # 欧几里得距离
        print("欧几里得距离:", path)

        # 标准的曼哈顿距离
        # print("标准的曼哈顿距离:", path)

        # 更倾向于向45度方向寻找的曼哈顿距离
        # print("更倾向于向45度方向寻找的曼哈顿距离:", path)

        # 更倾向于从起点指向终点方向寻找的曼哈顿距离
        # print("更倾向于从起点指向终点方向寻找的曼哈顿距离:", path)
        # 恢复原始的 sys.stdout
        sys.stdout = original_stdout
    # 文件输出完毕################################################################################
    # over #################################################################################################

    # visualization the path
    # Open the visualization_underground/my_path_in_London_railway.html to view the path, and your path is marked in red

    # plot_path(path, 'visualization_underground/my_shortest_path_in_London_railway.html', stations, underground_lines)

    # 所有关于距离的函数运行完再解除封印
    with open('output.txt', 'r') as f:
        # 读取文件的第一行
        first_line = f.readline()
        second_line = f.readline()
        third_line = f.readline()
        forth_line = f.readline()
        # 从字符串中提取列表部分，这里使用 eval 函数
        path = eval(first_line.split(':')[-1])
        path2 = eval(second_line.split(':')[-1])
        path3 = eval(second_line.split(':')[-1])
        path4 = eval(second_line.split(':')[-1])

    plot_overlap_path(path, path2, 'visualization_underground/my_shortest_path_in_London_railway.html',
                      stations, underground_lines)
